In [1]:
import pandas as pd
import numpy as np
train_identity = pd.read_csv('Data/train_identity.csv')
train_transaction = pd.read_csv('Data/train_transaction.csv')

In [2]:
#Storing variable names
identity_vars = list(train_identity)
transaction_vars = list(train_transaction)

#Storing id, fraud and separating from explanatory variables
trans_id = train_transaction['TransactionID']
fraud = train_transaction['isFraud']
x_trans = train_transaction.drop(['TransactionID','isFraud'],axis=1)

In [12]:
#Getting dummies from strings
strings = train_transaction.select_dtypes(include='object')
numerics = train_transaction.select_dtypes(exclude='object')
dummies = pd.get_dummies(strings)
x_trans = pd.concat([dummies, numerics],axis=1)
x_trans = x_trans.drop(['TransactionID','isFraud'],axis=1)

In [13]:
#Getting a count of NaNs
transaction_na_count = x_trans.isnull().sum()
transaction_na_prop = x_trans.isnull().sum()/x_trans.shape[0]*100
transaction_na = pd.concat([transaction_na_count, transaction_na_prop], axis=1)
transaction_na.columns = ['Count','Percentage']

In [14]:
#Calculating correlation coefficients with target 
cols = transaction_na.index.values[transaction_na['Percentage']<50]
corrs = train_transaction[x_cols].corrwith(fraud)

Just copying stuff from the EDA notebook, nothing new. What we're going to do now is:
1. Set up procedures for cross-validation and calculation of the ROC score
2. Train lasso and ridge models
3. Evaluate them using cross-validation

In [15]:
from sklearn.metrics import roc_auc_score
from sklearn import model_selection as ms
from sklearn.metrics import make_scorer
from sklearn.linear_model import Lasso

In [16]:
fraud_lasso = Lasso(alpha=0.1)

fraud_lasso.fit(X=x_trans,y=fraud)
#cv_scores = ms.cross_validate(fraud_lasso, x_trans,fraud, scoring = make_scorer(roc_auc_score), cv=5)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').